In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
#     df = pd.DataFrame(columns=common_words)
#     df['text_sentence'] = sentences[0]
#     df['text_source'] = sentences[1]
#     df.loc[:, common_words] = 0
    
    counts = np.zeros((len(sentences),len(common_words)), dtype=int)
   
    # Process each row, counting the occurrence of words in each sentence.
    text_sentences = list(sentences[0])
    for i, sentence in enumerate(text_sentences):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            
            counts[i, common_words.index(word)] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
            
    return counts

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = list(set(alicewords + persuasionwords))

In [6]:
# Create our data frame with features. This can take a while to run.
counts = bow_features(sentences, common_words)
counts

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700
Processing row 4800
Processing row 4900
Processing r

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
word_counts = pd.DataFrame(data=counts, columns=common_words)
word_counts['text_sentence'] = sentences[0]
word_counts['text_source'] = sentences[1]

## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [8]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

c:\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9877742946708464

Test set score: 0.8980263157894737


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

c:\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [10]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8846394984326019

Test set score: 0.8735902255639098


# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  



In [11]:
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

In [12]:
# fit and score svc
svc = SVC()
svc.fit(X_train, y_train)

c:\python37\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [13]:
svc.score(X_test, y_test)

0.6917293233082706

In [15]:
# fit and score naive bayes
nb = BernoulliNB()
nb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [16]:
nb.score(X_test, y_test)

0.9356203007518797

In [17]:
from sklearn.model_selection import cross_val_score

In [18]:
cross_val_score(nb, X, Y, cv=6)

array([0.91779279, 0.90632054, 0.94130926, 0.9221219 , 0.91873589,
       0.91196388])

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [19]:
hamlet = gutenberg.raw('shakespeare-hamlet.txt')

hamlet = re.sub(r'Chapter \d+', '', hamlet)
   
hamlet = text_cleaner(hamlet)

In [20]:
hamlet_doc = nlp(hamlet)

In [21]:
hamlet_sents = [[sent, "Shakespeare"] for sent in hamlet_doc.sents]

In [22]:
# Combine the sentences from the two novels into one data frame.
sentences2 = pd.DataFrame(alice_sents + hamlet_sents)
sentences2.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [23]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
hamletwords = bag_of_words(hamlet_doc)

# Combine bags to create a set of unique words.
common_words2 = list(set(alicewords + hamletwords))

In [24]:
# Create our data frame with features. This can take a while to run.
counts2 = bow_features(sentences2, common_words2)
counts2

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [25]:
word_counts2 = pd.DataFrame(data=counts2, columns=common_words2)
word_counts2['text_sentence'] = sentences2[0]
word_counts2['text_source'] = sentences2[1]

In [26]:
Y2 = word_counts2['text_source']
X2 = word_counts2.drop(['text_sentence','text_source'], 1)

In [27]:
nb = BernoulliNB()
cross_val_score(nb, X2, Y2, cv=6)

array([0.90585242, 0.89936306, 0.92993631, 0.91964286, 0.91836735,
       0.90816327])

I think the models presented in these examples are flawed because the features are engineered using information from test set. The features need to be engineered WITHOUT using information from the test set. The common words used as features need to be selected based on their frequency in the training set only. The sentences need to be split into training and test sets BEFORE the most common words are selected as features, not after.

The code below does this 

In [28]:
# split sentences2 df into training and test dataframes before identifying most common words
train_index = np.random.choice(len(sentences2), int(len(sentences2)*.75), replace=False)
train = sentences2.iloc[train_index]
test = sentences2.drop(train)

In [29]:
# Filter out punctuation and stop words.
allwords = []
for text in train[train[1]=='Carroll'][0]:
    allwords += [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]

# Return the most common words.
alice_words = [item[0] for item in Counter(allwords).most_common(2000)]

allwords = []
for text in train[train[1]=='Shakespeare'][0]:
    allwords += [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]

# Return the most common words.
hamlet_words = [item[0] for item in Counter(allwords).most_common(2000)]
    
# Combine bags to create a set of unique words.
common_words2 = list(set(alice_words + hamlet_words))

In [30]:
# Create our data frame with features. This can take a while to run.
counts2 = bow_features(sentences2, common_words2)
counts2

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
word_counts2 = pd.DataFrame(data=counts2, columns=common_words2)
word_counts2['text_sentence'] = sentences2[0]
word_counts2['text_source'] = sentences2[1]

In [32]:
X_train = word_counts2.iloc[train_index].drop(['text_sentence','text_source'], 1)
y_train = word_counts2.iloc[train_index]['text_source']

In [33]:
# fitnaive bayes
nb = BernoulliNB()
nb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [34]:
X_test = word_counts2.drop(train_index).drop(['text_sentence','text_source'], 1)
y_test = word_counts2.drop(train_index)['text_source']
nb.score(X_test, y_test)

0.923534409515718